# 5. 함수도  객체

- 함수도 1급 객체
- 함수도 다른 객체처럼 클래스를 가지고 있다.
- 함수도 다른 객체처럼 변수에 할당도 가능하다. 즉 매개변수로 전달이 가능하다.

## 5.1  함수도 클래스가 존재

In [1]:
def sum_all(args):
    return sum(args)

In [2]:
from types import FunctionType

In [3]:
sum_all.__class__ is FunctionType

True

## 5.2 다른 변수에 할당
- 1급 객체이므로 변수에 할당이 가능

### 함수를 다른 변수에 할당  

In [4]:
def origin_name():
    print("origin function")

In [5]:
new_name = origin_name 

In [6]:
new_name()

origin function


### 변수에 재할당은 레퍼런스 저장이므로 동일한 함수 

In [7]:
new_name is origin_name

True

In [8]:
new_name.__name__

'origin_name'

### `__qualname__`

함수가 생성될때 지정한 함수의 이름, 메소드인 경우에는 클래스의 이름을 포함

In [9]:
new_name.__qualname__

'origin_name'

## 매개변수에 초기 고정값 지정하기 

- 매개변수에 초기값을 지정하는 이유는 이 매개변수에 인자가 안들어와서 에러 없이 처리한다.
- 다른 언어는 함수를 오버로딩이 가능해서 여러 함수를 정의하는 것을 방지하기 위해 초기값을 지정하지만 파이썬은 여러 함수를 동시에 정의할 수 없어서 초기값만으로 여러 함수를 정의한 것 처럼 사용한다.

- `__defaults__`

In [10]:
def func_11(a=1, b=2, *, kwarg1="FOO", kwarg2="BAR", kwarg3="BAZ"):
    print(a, b)


In [11]:
func_11.__defaults__

(1, 2)

In [12]:
func_11.__kwdefaults__

{'kwarg1': 'FOO', 'kwarg2': 'BAR', 'kwarg3': 'BAZ'}

# 6.  익명함수  lambda 함수

- 이름없는 함수를 만든다.
- 이름없는 함수는 사용한 즉시 가비지컬렉션으로 사라짐
- 변수에 할당할 경우는 이름있는 함수로 지정해서 사용


## 6.1 익명함수 정의  및 실행

- lambda 키워도 지정
- 콜론 좌측은 매개변수 우측은 표현식 지정
- 문장을 표현식에 작성하면 예외를 발생시킴


### 익명함수는 매개변수 와 표현식을  콜론으로 구분한다.  반환값은 표현식이 결과

In [13]:
lambda x : x + 1

<function __main__.<lambda>(x)>

### 정의한 후에 바로 실행이 가능 

In [14]:
(lambda x : x + 1)(3)

4

### 익명함수와 함수는 동일하다. 

In [15]:
from types import LambdaType, FunctionType

In [16]:
LambdaType is FunctionType

True

## 6.2 람다 함수를 사용하는 이유

### 파이썬도 가비지 컬렉션으로 객체를 삭제하므로 약한 참조일 경우 별도로 관리가 가능하다

In [17]:
import weakref

In [18]:
def function1(x):
    x += 1
    return x

### 함수를 약한 참조로 등록하면 삭제하기 전까지 사용이 가능 

In [19]:
wkr_function = weakref.ref(function1)

In [20]:
wkr_function

<weakref at 0x7fdacaace9b0; to 'function' at 0x7fdacab3af80 (function1)>

In [21]:
function1(5)

6

In [22]:
function1(6)

7

### 익명 함수 삭제

In [23]:
del function1

In [24]:
wkr_function

<weakref at 0x7fdacaace9b0; dead>

### 재호출할 경우 삭제되어 예외발생

In [25]:
try :
    function1(60)
except Exception as e :
    print(e)

name 'function1' is not defined


### 람다함수는 약한 참조에 넣어도 한번 사용한 것으로 인식되어 삭제없이도 사라짐

In [26]:
wkr_lambda = weakref.ref(lambda x : x + 1)

In [27]:
wkr_lambda

<weakref at 0x7fdacaaf9290; dead>

# 7.  클로저 이해하기 

- 함수 내부에 있는 지역변수를 외부에서 갱신하도록 처리할 수 있는 환경을 만들어준다.


In [28]:
import weakref

###  외부 함수 내의 로컬변수가 내부함수를 통해  사용되어 함수함수 호출이후에도 사용이 되는 환경 

In [29]:
def func_c(a):
    wkr_a = weakref.ref(a)
    def inner():
        print(a)
        return wkr_a
    return inner

### 클래스 정의

In [30]:
class A:
    pass

### 함수에 객체 전달

In [31]:
inner = func_c(A())

### 반환된 내부 함수를 실행

In [32]:
inner()

<weakref at 0x7fdacaace770; to 'A' at 0x7fdacaacd490>

### 클로저에 정보는 `__clousre__` 속성에 정보를 확인한다

In [33]:
inner.__closure__[0].cell_contents

In [34]:
type(inner.__closure__[0].cell_contents)

__main__.A

# 8. 순수 함수, 합성함수, 재귀함수

## 순수함수와 비순수 함수

- 순수함수란 함수가 매개변수에 맞는 인자가 전달되면 항상 동일한 결과만 제공해야한다.
- 부수효과가 발생하면 비순수함수이다. 함수 이외의 변수를 변경하거나 다른 곳에 출력하는 것을 부수효과라고 한다.

### 항상 동일한 결과만 나온다

In [35]:
def add(x,y) :
    return x + y

In [36]:
add(10, 10)

20

In [37]:
add(10,10)

20

### 특정 전역변수의 값을 변경해서 동일한 결과가 나오지 않는다.

In [38]:
var = 100 

def add_(x,y) :
    global var
    var = x + y + var
    return x + y + var

In [39]:
add_(10,10)

140

In [40]:
add_(10,10)

160

## 합성함수

- 함수의 매개변수로 함수를 전달
- 함수의 반환값으로 함수를 전달

In [41]:
def func(parafunc) :
    return parafunc()

In [42]:
func(lambda : print("함수를 매개변수로 전달"))

함수를 매개변수로 전달


In [43]:
def func__() :
    return func

In [44]:
h = func__()

In [45]:
h

<function __main__.func(parafunc)>

In [46]:
h(lambda : print("함수를 매개변수로 전달"))

함수를 매개변수로 전달


## 재귀함수

- 함수 자신을 호출하고 함수의 결과와 다른 변수가 계산되어 처리하는 재귀함수
- 꼬리 재귀는 함수 자신만을 호출해서 처리하는 재귀함수
- 재귀함수의 특징은 순환문없이 함수를 호출해서 순환처리

### 순환문으로 팩토리얼 계산하기

In [47]:
def factorial_for(n):
    ret = 1
    for i in range(1, n+1):
        ret *= i
    return ret

In [48]:
factorial_for(5)

120

### 리듀스 함수를 사용해서 팩토리얼 계산하기

In [49]:
from functools import reduce

def factorial_reduce(n):
    return reduce(lambda x, y: x * y, range(1, n+1))

In [50]:
factorial_reduce(5)

120

### 재귀함수로 팩토리얼 계산하기

In [51]:
def factorial_recursive(n):
    return n * factorial_recursive(n-1) if n > 1 else 1

In [52]:
factorial_recursive(5)

120

In [53]:
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n - 1)

In [54]:
factorial(5)

120

### 꼬리 재귀는 계산결과를 함수의 매개변수로 처리한다.

In [55]:
def factorial_(n, result=1):
    if n == 0:
        return result
    else:
        return factorial_(n - 1, n * result)

In [56]:
factorial_(5)

120

## 팩토리 계산결과를 캐쉬해서 처리하기
- 재귀함수를 계산호출하면 함수 실행환경을 계속 스택에 구성
- 한번 처리한 것을 메모리에 저장한 후에 다시 호출할 때 있으면 조회한 값을 처리해서 함수 실행환경을 적게 구성

In [57]:
cache = {}

In [58]:
def factorial_rec_cache(n):
    global cache

    if n in cache:
        return cache[n]
    elif n <= 1:
        return 1
    else:
        cache[n] = n * factorial_rec_cache(n-1)
        return cache[n]

    return n * factorial_rec_cache(n-1) if n > 1 else 1

In [59]:
%%time
factorial_rec_cache(5)

CPU times: user 7 µs, sys: 1e+03 ns, total: 8 µs
Wall time: 9.78 µs


120

In [60]:
cache

{2: 2, 3: 6, 4: 24, 5: 120}

In [61]:
%%time
factorial_rec_cache(5)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.91 µs


120

In [62]:
%%time
factorial_rec_cache(20)

CPU times: user 15 µs, sys: 1e+03 ns, total: 16 µs
Wall time: 18.8 µs


2432902008176640000

In [63]:
%%time
factorial_rec_cache(20)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.96 µs


2432902008176640000